In [2]:
import os
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import LEDForConditionalGeneration, LEDConfig, Trainer, TrainingArguments
import torch.nn as nn


# Set the device to CUDA if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Verify CUDA availability
if not torch.cuda.is_available():
    raise SystemError("CUDA is not available. Please check your installation.")

# Load your saved .npy file
data = np.load('processed_tracks_data_10secs.npy', allow_pickle=True).item()
top_lvl_keys = list(data.keys())

sample_id = list(data.keys())[0]
sample_data = data[sample_id]
keys_for_sample = list(sample_data.keys())

VOCAB_SIZE = 1026
MAX_LENGTH = 3000  # Adjusted for maximum length

class MusicDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample_id = list(self.data.keys())[idx]
        sample = self.data[sample_id]

        vocal_audio_codes = sample['generation_data'].get('vocal', np.zeros((4, 750)))
        full_instrumental = sample['generation_data'].get('full_instrumental', np.zeros((4, 750)))
        positional_embedding = sample.get('positional_embedding', np.zeros((4, 750)))

        vocal_audio_codes = np.clip(vocal_audio_codes, 0, VOCAB_SIZE - 1)

        # Flatten and pad/truncate sequences
        vocal_audio_codes = np.pad(vocal_audio_codes.flatten(), (0, MAX_LENGTH - len(vocal_audio_codes)), 'constant', constant_values=(0, 0))[:MAX_LENGTH]
        full_instrumental = np.pad(full_instrumental.flatten(), (0, MAX_LENGTH - len(full_instrumental)), 'constant', constant_values=(0, 0))[:MAX_LENGTH]

        # Calculate the required padding length for positional embedding and apply padding
        padding_length = MAX_LENGTH - positional_embedding.shape[0]
        positional_embedding = np.pad(positional_embedding, ((0, padding_length), (0, 0)), mode='constant', constant_values=(0, 0))

        attention_mask = (vocal_audio_codes != 0).astype(int)
        return {
            'input_ids': torch.tensor(vocal_audio_codes, dtype=torch.long).to(device),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long).to(device),
            'labels': torch.tensor(full_instrumental, dtype=torch.long).to(device),
            'positional_embeddings': torch.tensor(positional_embedding, dtype=torch.float).to(device)
        }

def shift_tokens_right(input_ids, pad_token_id, decoder_start_token_id):
    """Shift input ids one token to the right, and wrap the last non pad token (usually <eos>)."""
    shifted_input_ids = input_ids.clone()
    shifted_input_ids[:, 1:] = input_ids[:, :-1].clone()
    shifted_input_ids[:, 0] = decoder_start_token_id

    # Replace possible -100 in the labels by `pad_token_id`
    shifted_input_ids.masked_fill_(shifted_input_ids == -100, pad_token_id)

    return shifted_input_ids

class CustomLEDForConditionalGeneration(LEDForConditionalGeneration):
    def __init__(self, config):
        super().__init__(config)
        self.projection = nn.Linear(config.d_model * 2, config.d_model)

    def forward(self, input_ids=None, attention_mask=None, labels=None, positional_embeddings=None, decoder_input_ids=None, **kwargs):
        input_embeds = self.led.encoder.embed_tokens(input_ids)
        embs_dim = positional_embeddings.shape[2]
        input_embeds = torch.cat((input_embeds[:, :, :-embs_dim], input_embeds[:, :, -embs_dim:] + positional_embeddings), dim=-1)

        # Handling cases where decoder_input_ids is not provided
        if decoder_input_ids is None:
            if input_ids is not None:
                # Shift input_ids to the right to create decoder_input_ids
                decoder_input_ids = shift_tokens_right(input_ids, self.config.pad_token_id, self.config.decoder_start_token_id)
            else:
                raise ValueError("decoder_input_ids and input_ids both cannot be None")

        return super().forward(inputs_embeds=input_embeds, attention_mask=attention_mask, labels=labels, decoder_input_ids=decoder_input_ids, **kwargs)


# Configuration for the model
config = LEDConfig(
    vocab_size=VOCAB_SIZE,
    decoder_start_token_id=VOCAB_SIZE - 2,
    pad_token_id=VOCAB_SIZE - 2,
    max_encoder_position_embeddings=MAX_LENGTH,
    max_decoder_position_embeddings=MAX_LENGTH,
    attention_window=[500] * 6,  # Increase number of layers
    encoder_layers=6,
    encoder_ffn_dim=1024,  # Increase the feedforward network size
    encoder_attention_heads=8,  # Increase the number of attention heads
    decoder_layers=6,
    decoder_ffn_dim=1024,
    decoder_attention_heads=8,
    use_cache=True,
    is_encoder_decoder=True,
    activation_function='gelu',
    d_model=128,
    dropout=0.0,  # Disable dropout
    init_std=0.02,
    eos_token_id=VOCAB_SIZE - 1,
    gradient_checkpointing=False
)

# Initialize the custom model
model = CustomLEDForConditionalGeneration(config=config).to(device)

# Create dataset and dataloader
dataset = MusicDataset(data)
overfit_dataset = torch.utils.data.Subset(dataset, [0])
subset_key = list(data.keys())[0]
print(f"The key of the subset dataset is: {subset_key}")

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=1,  # Adjust based on your GPU's memory capacity
    per_device_eval_batch_size=1,
    num_train_epochs=5000,
    weight_decay=0.00,
    save_total_limit=3,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
    fp16=True,  # Enable mixed precision training if your GPU supports it
    dataloader_pin_memory=False
)

# Custom data collator to handle positional embeddings
class DataCollatorWithPositionalEmbeddings:
    def __call__(self, batch):
        input_ids = torch.stack([item['input_ids'] for item in batch])
        attention_mask = torch.stack([item['attention_mask'] for item in batch])
        labels = torch.stack([item['labels'] for item in batch])
        positional_embeddings = torch.stack([item['positional_embeddings'] for item in batch])
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels,
            'positional_embeddings': positional_embeddings
        }

data_collator = DataCollatorWithPositionalEmbeddings()

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=overfit_dataset,
    eval_dataset=overfit_dataset,
    data_collator=data_collator
)

# Train the model
trainer.train()

# Save the model
trainer.save_model('./saved_model')

print("Model saved.")

The key of the subset dataset is: backdown_1_16bars


C:\Users\Ripple\AppData\Local\Programs\Python\Python39\lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


torch.Size([3000, 1026]) torch.Size([3000]) eta


Epoch,Training Loss,Validation Loss
1,No log,6.934358
2,No log,6.912484
3,No log,6.890659
4,No log,6.868884
5,No log,6.847150
6,No log,6.825482
7,No log,6.803900
8,No log,6.782458
9,No log,6.761232
10,6.858300,6.740329


torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([300

...

torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([300

torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([300

torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([3000]) eta
torch.Size([3000, 1026]) torch.Size([300

There were missing keys in the checkpoint model loaded: ['led.encoder.embed_tokens.weight', 'led.decoder.embed_tokens.weight', 'lm_head.weight'].


Model saved.


In [3]:
from transformers import LEDConfig

model_path = './saved_model'
model = CustomLEDForConditionalGeneration.from_pretrained(model_path).to(device)



def generate_full_instrumental(model, dataset, device):
    # Assuming the dataset has only one sample for overfitting evaluation
    sample = dataset[0]  # Get the first and only sample
    input_ids = sample['input_ids'].unsqueeze(0)  # Add batch dimension
    attention_mask = sample['attention_mask'].unsqueeze(0)  # Add batch dimension
    positional_embeddings = sample['positional_embeddings'].unsqueeze(0)  # Add batch dimension
    
    # Ensure everything is on the correct device
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    positional_embeddings = positional_embeddings.to(device)
    
    print(input_ids.shape, attention_mask.shape, positional_embeddings.shape)
    
    # Generate output
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Turn off gradients for validation, saves memory and computations
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, positional_embeddings=positional_embeddings)
        
        # Depending on your model's configuration, you may need to adjust how you extract the generated sequence
        generated_sequence = outputs.logits.argmax(dim=-1)  # Assuming logits are returned and we take the max logit for each timestep
    
    return generated_sequence

# Load the trained model (if not already loaded)
# model = CustomLEDForConditionalGeneration(config=config).to(device)
# model.load_state_dict(torch.load('path_to_trained_model.pth'))

# Generate the full instrumental from the model
generated_instrumental = generate_full_instrumental(model, overfit_dataset, device)

# To compare, get the true labels from the dataset
true_instrumental = overfit_dataset[0]['labels'].unsqueeze(0)

print(true_instrumental, generated_instrumental)
accuracy = (generated_instrumental == true_instrumental).float().mean()
print(f"Accuracy of reproduction: {accuracy.item() * 100:.2f}%")

torch.Size([1, 3000]) torch.Size([1, 3000]) torch.Size([1, 3000, 128])
tensor([[212,   0, 260,  ...,  56, 167, 474]], device='cuda:0') tensor([[212,   0, 260,  ...,  56, 167, 474]], device='cuda:0')
Accuracy of reproduction: 100.00%


In [8]:
# Now reshape the generated output to match the desired shape (4, seq_len)
num_parts = 4
seq_len = 750

# Reshape the generated sequence to match the original dimensions (4, seq_len)
reshaped_output = generated_instrumental.view(num_parts, seq_len).unsqueeze(0).unsqueeze(0)
original = true_instrumental.view(num_parts, seq_len).unsqueeze(0).unsqueeze(0)

# Print the reshaped output and its shape
print("Generated Instrumental Shape after reshaping:", reshaped_output.shape)

Generated Instrumental Shape after reshaping: torch.Size([1, 1, 4, 750])


In [6]:
from transformers import AutoProcessor, EncodecModel, EncodecConfig
processor = AutoProcessor.from_pretrained("facebook/encodec_24khz")
from IPython.display import display, Audio
        
model = EncodecModel.from_pretrained("facebook/encodec_24khz")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


EncodecModel(
  (encoder): EncodecEncoder(
    (layers): ModuleList(
      (0): EncodecConv1d(
        (conv): Conv1d(1, 32, kernel_size=(7,), stride=(1,))
      )
      (1): EncodecResnetBlock(
        (block): ModuleList(
          (0): ELU(alpha=1.0)
          (1): EncodecConv1d(
            (conv): Conv1d(32, 16, kernel_size=(3,), stride=(1,))
          )
          (2): ELU(alpha=1.0)
          (3): EncodecConv1d(
            (conv): Conv1d(16, 32, kernel_size=(1,), stride=(1,))
          )
        )
        (shortcut): EncodecConv1d(
          (conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
        )
      )
      (2): ELU(alpha=1.0)
      (3): EncodecConv1d(
        (conv): Conv1d(32, 64, kernel_size=(4,), stride=(2,))
      )
      (4): EncodecResnetBlock(
        (block): ModuleList(
          (0): ELU(alpha=1.0)
          (1): EncodecConv1d(
            (conv): Conv1d(64, 32, kernel_size=(3,), stride=(1,))
          )
          (2): ELU(alpha=1.0)
          (3): EncodecC

In [7]:
audio_values = model.decode(reshaped_output, [None])[0]
# Convert audio tensor to numpy array
audio_np = audio_values.cpu().squeeze().detach().numpy()
audio_widget = Audio(data=audio_np, rate=processor.sampling_rate)
display(audio_widget)

In [9]:
audio_values = model.decode(original, [None])[0]
# Convert audio tensor to numpy array
audio_np = audio_values.cpu().squeeze().detach().numpy()
audio_widget = Audio(data=audio_np, rate=processor.sampling_rate)
display(audio_widget)